In [115]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
df=[]
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        df.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [133]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [117]:
df

['/kaggle/input/healthy-diet-recipes-a-comprehensive-dataset/All_Diets.csv',
 '/kaggle/input/healthy-diet-recipes-a-comprehensive-dataset/mediterranean.csv',
 '/kaggle/input/healthy-diet-recipes-a-comprehensive-dataset/paleo.csv',
 '/kaggle/input/healthy-diet-recipes-a-comprehensive-dataset/vegan.csv',
 '/kaggle/input/healthy-diet-recipes-a-comprehensive-dataset/keto.csv',
 '/kaggle/input/healthy-diet-recipes-a-comprehensive-dataset/dash.csv']

In [118]:
new_df1=[]
for i in df:
    new_df1.append(pd.read_csv(i))

# # data

In [119]:
new_df=pd.concat(new_df1,axis=0)
new_df

,Diet_type,Recipe_name,Cuisine_type,Protein(g),Carbs(g),Fat(g),Extraction_day,Extraction_time
0,paleo,Bone Broth From 'Nom Nom Paleo',american,5.22,1.29,3.20,2022-10-16,17:20:09
1,paleo,"Paleo Effect Asian-Glazed Pork Sides, A Sweet ...",south east asian,181.55,28.62,146.14,2022-10-16,17:20:09
2,paleo,Paleo Pumpkin Pie,american,30.91,302.59,96.76,2022-10-16,17:20:09
3,paleo,Strawberry Guacamole recipes,mexican,9.62,75.78,59.89,2022-10-16,17:20:09
4,paleo,"Asian Cauliflower Fried ""Rice"" From 'Nom Nom P...",chinese,39.84,54.08,71.55,2022-10-16,17:20:09
...,...,...,...,...,...,...,...,...
1740,dash,Brown Butter-Sunchoke Soup With Brussels Sprou...,italian,85.20,288.14,137.15,2022-10-16,20:40:44
1741,dash,Make-Your-Own-Salad,american,141.98,123.18,323.50,2022-10-16,20:40:44
1742,dash,Luis Buñuel Dry Martini,world,0.01,0.39,0.00,2022-10-16,20:40:44
1743,dash,Cornflake Semi-Fried Chicken Tenders,american,155.38,239.88,260.84,2022-10-16,20:40:44


In [120]:
# new_df.sample(3)
new_df["Date"]=pd.to_datetime(new_df["Extraction_day"])
new_df["Year"]=new_df["Date"].dt.year
new_df["Month"]=new_df["Date"].dt.month
new_df["Day"]=new_df["Date"].dt.day
# new_df["Extraction_time"]=pd.to_datetime(new_df["Extraction_time"]).dt.time
# new_df["DayofWeek"]=new_df["Extraction_day"].dt.dayofweek
new_df["Hour"]=pd.to_datetime(new_df["Extraction_time"],format='%H:%M:%S').dt.hour
new_df=new_df.drop(columns=["Extraction_day","Extraction_time","Date"])

In [121]:
new_df

,Diet_type,Recipe_name,Cuisine_type,Protein(g),Carbs(g),Fat(g),Year,Month,Day,Hour
0,paleo,Bone Broth From 'Nom Nom Paleo',american,5.22,1.29,3.20,2022,10,16,17
1,paleo,"Paleo Effect Asian-Glazed Pork Sides, A Sweet ...",south east asian,181.55,28.62,146.14,2022,10,16,17
2,paleo,Paleo Pumpkin Pie,american,30.91,302.59,96.76,2022,10,16,17
3,paleo,Strawberry Guacamole recipes,mexican,9.62,75.78,59.89,2022,10,16,17
4,paleo,"Asian Cauliflower Fried ""Rice"" From 'Nom Nom P...",chinese,39.84,54.08,71.55,2022,10,16,17
...,...,...,...,...,...,...,...,...,...,...
1740,dash,Brown Butter-Sunchoke Soup With Brussels Sprou...,italian,85.20,288.14,137.15,2022,10,16,20
1741,dash,Make-Your-Own-Salad,american,141.98,123.18,323.50,2022,10,16,20
1742,dash,Luis Buñuel Dry Martini,world,0.01,0.39,0.00,2022,10,16,20
1743,dash,Cornflake Semi-Fried Chicken Tenders,american,155.38,239.88,260.84,2022,10,16,20


# # One hot encoder

In [122]:
category_item=["Diet_type","Cuisine_type"]
one_hot=OneHotEncoder()
one=one_hot.fit_transform(new_df[category_item])
one=pd.DataFrame(one.toarray(),columns=one_hot.get_feature_names_out(category_item))

In [123]:
one.sample()

,Diet_type_dash,Diet_type_keto,Diet_type_mediterranean,Diet_type_paleo,Diet_type_vegan,Cuisine_type_american,Cuisine_type_asian,Cuisine_type_british,Cuisine_type_caribbean,Cuisine_type_central europe,...,Cuisine_type_italian,Cuisine_type_japanese,Cuisine_type_kosher,Cuisine_type_mediterranean,Cuisine_type_mexican,Cuisine_type_middle eastern,Cuisine_type_nordic,Cuisine_type_south american,Cuisine_type_south east asian,Cuisine_type_world
3116,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# # TfidfVectorizer

In [124]:
category_item_tfid="Recipe_name"
tfidf=TfidfVectorizer()
t=tfidf.fit_transform(new_df["Recipe_name"])
t=t.toarray()
t=pd.DataFrame(t,columns=tfidf.get_feature_names_out())


In [125]:
t

,10,100,101,12th,131,15,151,20,2017,30,...,zoodle,zoodles,zool,zucchini,zuk,zuppa,éclairs,élysées,伊麵,蛋花湯
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# # concat

In [129]:
new_df = new_df.reset_index(drop=True)
concaty=pd.concat([new_df.drop(columns=["Diet_type","Recipe_name","Cuisine_type"]),one,t],axis=1)
concaty=concaty.drop(columns=["Protein(g)"])

In [130]:
concaty


,Carbs(g),Fat(g),Year,Month,Day,Hour,Diet_type_dash,Diet_type_keto,Diet_type_mediterranean,Diet_type_paleo,...,zoodle,zoodles,zool,zucchini,zuk,zuppa,éclairs,élysées,伊麵,蛋花湯
0,1.29,3.20,2022,10,16,17,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,28.62,146.14,2022,10,16,17,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,302.59,96.76,2022,10,16,17,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,75.78,59.89,2022,10,16,17,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,54.08,71.55,2022,10,16,17,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15607,288.14,137.15,2022,10,16,20,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15608,123.18,323.50,2022,10,16,20,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15609,0.39,0.00,2022,10,16,20,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15610,239.88,260.84,2022,10,16,20,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [128]:
print(new_df.index.duplicated().any())  # Check if there are duplicate indices
print(one.index.duplicated().any())     # Check if there are duplicate indices
print(pd.DataFrame(t).index.duplicated().any())  # Check if there are duplicate indices


False
False
False


In [131]:
Y=new_df[["Protein(g)"]]

In [135]:
X_train,X_test,Y_train,Y_test=train_test_split(concaty,Y,test_size=0.2,random_state=42)
random_reg=RandomForestRegressor(n_estimators=50)
random_reg.fit(X_train,Y_train)
random_reg.score(X_test,Y_test)

/tmp/ipykernel_33/3090624385.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_reg.fit(X_train,Y_train)


0.8466916616000854